In [1]:
!wget -q https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/school.json \
         https://lfs.aminer.cn/misc/moocdata/data/mooccube2/entities/course.json \
         https://lfs.aminer.cn/misc/moocdata/data/mooccube2/relations/course-school.txt

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('preprocessing data').getOrCreate()

In [14]:
from pyspark.sql.functions import col, trim, concat_ws, lit, split
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [4]:
course = spark.read.options(header=True, inferSchema=True).json('course.json')
course.show(5)

+-------------------------------------+------------------------------+--------+--------------------------+-------------+--------------------+
|                                about|                         field|      id|                      name|prerequisites|            resource|
+-------------------------------------+------------------------------+--------+--------------------------+-------------+--------------------+
|通过老师导读，同学们可深入这一经典...|        [历史学, 中国语言文学]|C_584313|          《资治通鉴》导读|             |[{1.1.1, V_849, [...|
|本课程是理工科的一门数学基础课，系...|[应用经济学, 数学, 物理学, ...|C_584329|微积分——极限理论与一元函数|             |[{1.1.1, V_1350, ...|
|掌握基本的摄影技能，了解图片新闻的...|          [艺术学, 新闻传播学]|C_584381|                  新闻摄影|             |[{1.1.1, V_1800, ...|
|  最有趣的理论+最有用的算法=不得不...|            [计算机科学与技术]|C_597208|      数据挖掘：理论与算法|             |[{1.1.1, V_2961, ...|
|大学计算机课程将以计算思维为导向，...|                            []|C_597225|                大学计算机|             |[{1.1.1, V_4596, ...|
+--------------------

In [5]:
school = spark.read.options(header=True, inferSchema=True).json('school.json')
school.show(5)

+-----------------------------------+---+----------------------+--------+-------------------+----+
|                              about| id|                 motto|    name|            name_en|sign|
+-----------------------------------+---+----------------------+--------+-------------------+----+
|简称“清华”，由中华人民共和国教育...|S_1|     自强不息,厚德载物|清华大学|Tsinghua University| thu|
|          北京大学（Peking Unive...|S_2|博学、审问、慎思、明辨|北京大学|  Peking University| PKU|
|          武汉大学（Wuhan Univer...|S_3|   自强 弘毅 求是 拓新|武汉大学|   Wuhan University| whu|
|          苏州大学（Soochow Univ...|S_4|养天地正气，法古今完人|苏州大学| Soochow University|suda|
|          四川大学（Sichuan Univ...|S_5|                      |四川大学| Sichuan University| scu|
+-----------------------------------+---+----------------------+--------+-------------------+----+
only showing top 5 rows



In [6]:
course_school = spark.read.options(header=True, inferSchema=True).text('course-school.txt')
course_school.show(5)

+-------------+
|        value|
+-------------+
|C_375629\tS_1|
|C_375775\tS_1|
|C_375778\tS_1|
|C_584313\tS_1|
|C_584329\tS_1|
+-------------+
only showing top 5 rows



In [7]:
# Split course and school id from course_school
course_school = course_school.withColumn("course_id", split(course_school['value'], '\t')[0]) \
                              .withColumn("school_id", split(course_school['value'], '\t')[1]) \
                              .drop('value')
course_school.show(5)

+---------+---------+
|course_id|school_id|
+---------+---------+
| C_375629|      S_1|
| C_375775|      S_1|
| C_375778|      S_1|
| C_584313|      S_1|
| C_584329|      S_1|
+---------+---------+
only showing top 5 rows



In [8]:
course_school.summary().show()

+-------+---------+---------+
|summary|course_id|school_id|
+-------+---------+---------+
|  count|     3983|     3983|
|   mean|     NULL|     NULL|
| stddev|     NULL|     NULL|
|    min|C_1017355|      S_1|
|    25%|     NULL|     NULL|
|    50%|     NULL|     NULL|
|    75%|     NULL|     NULL|
|    max| C_956450|     S_99|
+-------+---------+---------+



### 1. Filter

In [9]:
# Join with course to filter the valid course_id
valid_course_school = course_school.join(
    course,
    course_school.course_id == course.id,
    'inner'
).select(['course_id', 'school_id'])
valid_course_school.show(5)

+---------+---------+
|course_id|school_id|
+---------+---------+
| C_584313|      S_1|
| C_584329|      S_1|
| C_584381|      S_1|
| C_597208|      S_1|
| C_597225|      S_6|
+---------+---------+
only showing top 5 rows



In [10]:
# Join valid_course_school with school to filter the valid school_id
fully_valid_course_school = valid_course_school.join(
    school,
    valid_course_school.school_id == school.id,
    'inner'
).select(['course_id', 'school_id'])
fully_valid_course_school.show(5)

+---------+---------+
|course_id|school_id|
+---------+---------+
| C_584313|      S_1|
| C_584329|      S_1|
| C_584381|      S_1|
| C_597208|      S_1|
| C_597225|      S_6|
+---------+---------+
only showing top 5 rows



In [11]:
print("Số lượng dòng dữ liệu hợp lệ:", fully_valid_course_school.count())
print("Số lượng dòng dữ liệu không hợp lệ:", course_school.count() - fully_valid_course_school.count())

Số lượng dòng dữ liệu hợp lệ: 3605
Số lượng dòng dữ liệu không hợp lệ: 378


### 2. Mapping

### School

In [13]:
valid_schools = fully_valid_course_school.select("school_id").distinct()
valid_schools.show()

+---------+
|school_id|
+---------+
|     S_21|
|    S_316|
|    S_138|
|    S_319|
|    S_228|
|    S_486|
|    S_379|
|    S_244|
|    S_482|
|    S_497|
|     S_14|
|     S_64|
|    S_477|
|     S_34|
|     S_83|
|    S_146|
|    S_220|
|    S_131|
|    S_130|
|    S_249|
+---------+
only showing top 20 rows



In [15]:
# Add a new column with numbers starting from 1
window_spec = Window.orderBy("school_id")
df_mapped = valid_schools.withColumn("mapped_id", row_number().over(window_spec) + 229196)

In [16]:
# Select only the id and the new number
mapping_df = df_mapped.select(valid_schools["school_id"].alias("original_id"), "mapped_id")
mapping_df.tail(5)

[Row(original_id='S_964', mapped_id=229612),
 Row(original_id='S_97', mapped_id=229613),
 Row(original_id='S_971', mapped_id=229614),
 Row(original_id='S_98', mapped_id=229615),
 Row(original_id='S_99', mapped_id=229616)]

In [17]:
# Save as a single .txt file with two columns (CSV format)
mapping_df.coalesce(1).write \
    .option("header", True) \
    .option("delimiter", ",") \
    .mode("overwrite") \
    .csv("output/mapping_txt")

### Course-School

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
course_mapping = spark.read.options(header=True, inferSchema=True).csv('/content/drive/MyDrive/Big Data/Output/course_mapping.csv')
school_mapping = spark.read.options(header=True, inferSchema=True).csv('/content/drive/MyDrive/Big Data/Output/school.csv')

In [20]:
# Show mapping data
print("Course Mapping:")
course_mapping.show(10)
print("School Mapping:")
school_mapping.show(10)

Course Mapping:
+-----------+---------+
|original_id|mapped_id|
+-----------+---------+
|  C_1017355|        0|
|  C_1017419|        1|
|  C_1025064|        2|
|  C_1025076|        3|
|  C_1025079|        4|
|  C_1073350|        5|
|  C_1123814|        6|
|  C_1123848|        7|
|  C_1123944|        8|
|  C_1123979|        9|
+-----------+---------+
only showing top 10 rows

School Mapping:
+-----------+---------+
|original_id|mapped_id|
+-----------+---------+
|        S_1|   229197|
|       S_10|   229198|
|      S_100|   229199|
|     S_1006|   229200|
|      S_101|   229201|
|      S_102|   229202|
|      S_103|   229203|
|     S_1031|   229204|
|      S_104|   229205|
|      S_105|   229206|
+-----------+---------+
only showing top 10 rows



In [21]:
# Bước 1: Map school_id từ original sang mapped
school_mapped = fully_valid_course_school.join(
    school_mapping,
    fully_valid_course_school.school_id == school_mapping.original_id,
    'inner'
).select(
    school_mapping.mapped_id.alias("mapped_school_id"),
    fully_valid_course_school.course_id
)
school_mapped.show(5)

+----------------+---------+
|mapped_school_id|course_id|
+----------------+---------+
|          229197| C_584313|
|          229197| C_584329|
|          229197| C_584381|
|          229197| C_597208|
|          229528| C_597225|
+----------------+---------+
only showing top 5 rows



In [22]:
# Bước 2: Map course_id từ original sang mapped
course_mapped = school_mapped.join(
    course_mapping,
    school_mapped.course_id == course_mapping.original_id,
    "inner"
).select(
    school_mapped.mapped_school_id,
    course_mapping.mapped_id.alias("mapped_course_id")
)
course_mapped.show(5)

+----------------+----------------+
|mapped_school_id|mapped_course_id|
+----------------+----------------+
|          229197|            1158|
|          229197|            1159|
|          229197|            1160|
|          229528|            1161|
|          229197|            1162|
+----------------+----------------+
only showing top 5 rows



In [23]:
# Bước 3: Tạo chuỗi text theo format yêu cầu: course_id 0 school_id (0 là mối quan hệ teacher)
output_df = course_mapped.select(
    concat_ws(" ",
        col("mapped_course_id"),
              lit(3),
        col("mapped_school_id")
    ).alias("output_line")
)

In [24]:
output_df.show(5)

+-------------+
|  output_line|
+-------------+
|1158 3 229197|
|1159 3 229197|
|1160 3 229197|
|1161 3 229528|
|1162 3 229197|
+-------------+
only showing top 5 rows



In [25]:
output_df.select("output_line").coalesce(1).write.mode("overwrite").text("course_school")